In [1]:
import numpy as np

from sktime.datasets import load_UCR_UEA_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from utils.mlflow_saving import mlflow_save_result
from utils.read import reshape_new_to_old_format, reshape_old_to_new_format


# TODO: Verbose
# TODO: Feature unit-test:
# TODO: Describe Function: Get Experiment class information
class Experiment:
    def __init__(
        self,
        clasifier=None,
        dataset=None,
        augment={"name": str(None), "params": str(None)},
        verbose=0,
    ):
        self.clasifier = clasifier
        self.dataset = dataset
        self.augment = augment
        self.y_pred = None
        self.evaluation_metric = {}
        self.verbose = verbose

    def load_UCR_dataset(self, dataset: str = None):
        dataset_name = self.dataset if self.dataset else dataset
        x_train, y_train = load_UCR_UEA_dataset(
            dataset_name, split="train", return_X_y=True, return_type="numpy2D"
        )
        x_test, y_test = load_UCR_UEA_dataset(
            dataset_name, split="test", return_X_y=True, return_type="numpy2D"
        )
        # TODO: Data Inconsistency with interface
        self.dataset = {
            "name": dataset_name,
            "x_train": x_train,
            "y_train": y_train,
            "x_test": x_test,
            "y_test": y_test,
        }

    def augmentation(self):
        if self.augment["name"] == "None":
            return
        x_train = reshape_new_to_old_format(self.dataset["x_train"])
        x_test = reshape_new_to_old_format(self.dataset["x_test"])

        if self.augment.get('enter_label') == True:
            x_train_aug = self.augment["function"](x_train, self.dataset["y_train"], **self.augment["params"])
            x_test_aug = self.augment["function"](x_test, self.dataset["y_test"], **self.augment["params"])

        else:
            x_train_aug = self.augment["function"](x_train, **self.augment["params"])
            x_test_aug = self.augment["function"](x_test, **self.augment["params"])

        # Concat original or not
        if self.augment.get('concat_original') == True:
            x_train_aug = np.concatenate((x_train_aug, x_train), axis=0)
            x_test_aug = np.concatenate((x_test_aug, x_test), axis=0)
            self.dataset["y_train"] = np.concatenate((self.dataset["y_train"], self.dataset["y_train"]), axis=0)
            self.dataset["y_test"] = np.concatenate((self.dataset["y_test"], self.dataset["y_test"]), axis=0)

        self.dataset["x_train_aug"] = reshape_old_to_new_format(x_train_aug)
        self.dataset["x_test_aug"] = reshape_old_to_new_format(x_test_aug)

    def train_classier(self):
        if (self.dataset.get('x_train_aug').all() == None) or (self.dataset.get('x_test_aug').all() == None):
            self.clasifier["function"].fit(self.dataset["x_train"], self.dataset["y_train"])
        else:
            print(f'Training Shape: {self.dataset["x_train_aug"].shape}, {self.dataset["y_train"].shape}')
            self.clasifier["function"].fit(self.dataset["x_train_aug"], self.dataset["y_train"])

    def predict(self):
        self.y_pred = self.clasifier["function"].predict(self.dataset["x_test_aug"])

    def evaluate(self):
        self.evaluation_metric["accuracy"] = accuracy_score(
            self.dataset["y_test"], self.y_pred
        )
        (
            self.evaluation_metric["precision"],
            self.evaluation_metric["recall"],
            self.evaluation_metric["fbeta"],
            self.evaluation_metric["support"],
        ) = precision_recall_fscore_support(self.dataset["y_test"], self.y_pred)

    def save_result(self):
        mlflow_save_result(
            {"accuracy": self.evaluation_metric["accuracy"]},
            {"model": self.clasifier["name"]},
            {
                "dataset": self.dataset["name"],
                "datapoint_shape": str(self.dataset["x_train_aug"].shape)
                + "x"
                + str(self.dataset["x_test"].shape),
                "number_of_class": len(set(list(self.dataset["y_test"]))),
            },
            {
                "augmentation": {
                    "name": self.augment["name"],
                    "params": self.augment["params"],
                },
            },
            # [{"data": self.augment["params"], "file_name": "dict/augmentation.json"}],
        )
        print(f'{self.clasifier["name"]}, {self.dataset["name"]}, ({self.augment["name"]}, {self.augment["params"]}) DONE')

    def run_all(self):
        self.load_UCR_dataset()
        self.augmentation()
        self.train_classier()
        self.predict()
        self.evaluate()
        # self.save_result()

In [3]:
import warnings
warnings.filterwarnings("ignore")

# from experiment.Experiment import Experiment
# from experiment.ExperimentSet import ExperimentSet
from utils.read import read_UCR_dataset_name

from sktime.classification.kernel_based import RocketClassifier

In [4]:
from time_series_augmentation.utils.pun_augmentation import window_slice

variation = 'minirocket'
classifer = {'name':variation, 'function': RocketClassifier(rocket_transform=variation)}
dataset = 'ScreenType'
augment = {'name': 'window_slice_py', 'function': window_slice,'params': {}, 'concat_original': True}
experiments = Experiment(classifer, dataset, augment)
experiments.run_all()

Training Shape: (750, 1, 720), f(750,)


In [5]:
experiments.evaluation_metric

{'accuracy': 0.43066666666666664,
 'precision': array([0.3930131 , 0.40248963, 0.48571429]),
 'recall': array([0.36 , 0.388, 0.544]),
 'fbeta': array([0.37578288, 0.39511202, 0.51320755]),
 'support': array([250, 250, 250])}

In [6]:
from time_series_augmentation.utils.pun_augmentation import window_slice

variation = 'minirocket'
classifer = {'name':variation, 'function': RocketClassifier(rocket_transform=variation)}
dataset = 'ScreenType'
augment = {'name': 'window_slice_py', 'function': window_slice,'params': {}, 'concat_original': False}
experiments = Experiment(classifer, dataset, augment)
experiments.run_all()

Training Shape: (375, 1, 720), f(375,)


In [7]:
experiments.evaluation_metric

{'accuracy': 0.45866666666666667,
 'precision': array([0.43410853, 0.4368932 , 0.4965035 ]),
 'recall': array([0.448, 0.36 , 0.568]),
 'fbeta': array([0.44094488, 0.39473684, 0.52985075]),
 'support': array([125, 125, 125])}